In [1]:
import os
import csv
from countries_languages import country_to_language

import pandas as pd
from difflib import get_close_matches
import numpy as np
import re
from transliterate import translit
from unidecode import unidecode
import Levenshtein
import requests
import bs4
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
from datetime import datetime

from urllib.parse import urlsplit

import wikipediaapi

import wikipedia

from urllib.parse import urlparse

import difflib

from datetime import datetime

import calendar

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import AssemblyHelpers
#from AssemblyHelpers import find_money_info_from_name


def remove_accents(text):
    return unidecode(text)

def get_country_code(country):
    matching_rows = countries_codes.loc[countries_codes['Country'] == country, ' Code']
    return matching_rows.iloc[0] if not matching_rows.empty else None

def load_csv_dataset(file_path):
    return pd.read_csv(file_path)

#PLAYER DATA - MARKET VALUE AND SALARY
leagues_salary = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1//CSVs we use often/latest_capology_data_money_fixed.csv')

leagues_value = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/MarketValuesComplete.csv')


leagues_value_large = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/most_updated_transfermarkt_dataset.csv')

#COUNTRY CODES
countries_codes = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/countries_and_codes.csv')

#FIFA RANKINGS
fifa_rankings = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/fifa_ranking-2023-10-26 - fifa_ranking-2023-07-20.csv')

#MATCH DATA - LINEUPS AND RESULTS
# lineups_data = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/newest_lineups_data_march_24.csv')
lineups_data = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/newest_lineups_data_april_29.csv')
results_data = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/newest_results_data_march_24.csv')


leagues_value['Accents Removed'] = leagues_value['Name'].apply(lambda x: unidecode(x))
leagues_value['Team 1 Code'] = leagues_value['Nationality'].apply(get_country_code)

countries_codes['Accents Removed'] = countries_codes['Country'].apply(remove_accents)




/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/AssemblyHelpers.py:54: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)
/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_50468/2775728346.py:62: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  leagues_value_large = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/CSVs we use often/most_updated_transfermarkt_dataset.csv')


In [16]:
combined_df = load_csv_dataset('combined_DF_terrikon_removed.csv')
combined_df = combined_df.drop(columns={'Unnamed: 0'})
combined_df = combined_df[combined_df['Competition'] != 'Copa America 2016']

combined_df['Competition General'] = ''

for index, row in combined_df.iterrows():
    if 'World Cup Qualifiers, Europe' in row['Competition']:
        combined_df.at[index, 'Competition General'] = 'World Cup Qualifiers, Europe'
    if 'World Cup Qualifiers, Asia' in row['Competition']:
        combined_df.at[index, 'Competition General'] = 'World Cup Qualifiers, Asia'
    if 'World Cup Qualifiers, Africa' in row['Competition']:
        combined_df.at[index, 'Competition General'] = 'World Cup Qualifiers, Africa'
    if 'CONCACAF' in row['Competition']:
        combined_df.at[index, 'Competition General'] = 'World Cup Qualifiers, CONCACAF'
    if 'CONMEBOL' in row['Competition']:
        combined_df.at[index, 'Competition General'] = 'World Cup Qualifiers, CONMEBOL'
    if 'African Cup of Nations' in row['Competition']:
        combined_df.at[index, 'Competition General'] = 'African Cup of Nations'
    if 'Olympics' in row['Competition']:
        combined_df.at[index, 'Competition General'] = 'Olympics'
    if 'Gold Cup' in row['Competition']:
        combined_df.at[index, 'Competition General'] = 'Gold Cup'
    if 'World Cup Intercontinental Playoffs' in row['Competition']:
        combined_df.at[index, 'Competition General'] = 'World Cup Intercontinental Playoffs'
    if 'U-20 World Cup' in row['Competition']:
        combined_df.at[index, 'Competition General'] = 'U-20 World Cup'

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_50468/2775728346.py:54: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [25]:
# Create an empty dictionary to store the unique combinations of 'Match' and 'Date'
match_date_dict = {}
unique_number = 0

# Iterate over each unique value in the 'Competition General' column
for comp_general in combined_df['Competition General'].unique():
    # Filter the DataFrame for rows where 'Competition General' matches the current value
    all_matches_this_comp = combined_df[combined_df['Competition General'] == comp_general]
    
    # Iterate over each row in the filtered DataFrame
    for index, row in all_matches_this_comp.iterrows():
        # Create a unique key using the combination of 'Match' and 'Date'
        match_date_key = (row['Match'], row['Date'])
        
        # If the key doesn't exist in the dictionary, add it with a unique number
        if match_date_key not in match_date_dict:
            match_date_dict[match_date_key] = f"{comp_general}_{unique_number}"
            unique_number += 1

In [32]:
combined_df = combined_df.drop(columns={'Match ID'})
combined_df['Match ID'] = ''

In [33]:
for index, row in combined_df.iterrows():
    this_match = row['Match']
    this_date = row['Date']

    this_key = (this_match, this_date)

    if this_key in match_date_dict.keys():
        correct_match_ID = match_date_dict[this_key]
        combined_df.at[index, 'Match ID'] = correct_match_ID
    else:
        print(index)

In [34]:
combined_df

,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case,Competition General,Match ID
0,"World Cup Qualifiers, Europe, 2018",Luxembourg vs France,25-03-2017,Luxembourg,LU,2017,SUCCESS,Anthony Moris,400000.0,False,working,False,Anthony Moris,A Moris,single,"World Cup Qualifiers, Europe","World Cup Qualifiers, Europe_0"
1,"World Cup Qualifiers, Europe, 2018",Luxembourg vs France,25-03-2017,Luxembourg,LU,2017,SUCCESS,Laurent Jans,1200000.0,False,working,False,Laurent Jans,L Jans,single,"World Cup Qualifiers, Europe","World Cup Qualifiers, Europe_0"
2,"World Cup Qualifiers, Europe, 2018",Luxembourg vs France,25-03-2017,Luxembourg,LU,2017,SUCCESS,Kevin Malget,150000.0,False,working,False,Kevin Malget,K Malget,single,"World Cup Qualifiers, Europe","World Cup Qualifiers, Europe_0"
3,"World Cup Qualifiers, Europe, 2018",Luxembourg vs France,25-03-2017,Luxembourg,LU,2017,SUCCESS,Mario Mutsch,125000.0,False,working,False,Mario Mutsch,M Mutsch,single,"World Cup Qualifiers, Europe","World Cup Qualifiers, Europe_0"
4,"World Cup Qualifiers, Europe, 2018",Luxembourg vs France,25-03-2017,Luxembourg,LU,2017,SUCCESS,Chris Philipps,1250000.0,False,working,False,Chris Philipps,C Philipps,single,"World Cup Qualifiers, Europe","World Cup Qualifiers, Europe_0"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57624,"Olympics, 2016",Germany vs South Korea,2016-08-07,South Korea,KR,2016,SUCCESS,Kyu-baek Choi,350000.0,False,Found in Curr Season,False,Kyu-baek Choi,Choi Kyu-baek,single,Olympics,Olympics_1943
57625,"Olympics, 2020",Egypt vs Argentina,2021-07-25,Argentina,AR,2021,SUCCESS,Claudio Nicolás Bravo,2000000.0,False,Found in Curr Season,False,Claudio Nicolás Bravo,Claudio Bravo,single,Olympics,Olympics_1974
57626,"Olympics, 2020",Spain vs Argentina,2021-07-28,Argentina,AR,2021,SUCCESS,Claudio Nicolás Bravo,2000000.0,False,Found in Curr Season,False,Claudio Nicolás Bravo,Claudio Bravo,single,Olympics,Olympics_1977
57627,"Olympics, 2020",Australia vs Egypt,2021-07-28,Australia,AU,2021,SUCCESS,Dylan Pierias,350000.0,False,Found in Curr Season,False,Dylan Pierias,Dylan Pierias,single,Olympics,Olympics_1976


In [50]:
failed_df = load_csv_dataset('failed_df_terrikon_removed.csv')
failed_df = failed_df[failed_df['Competition'] != 'Copa America 2016']
failed_df = failed_df.drop(columns={'Match ID'})
failed_df['Match ID'] = ''
failed_df['Competition General'] = ''

for index, row in failed_df.iterrows():
    if 'World Cup Qualifiers, Europe' in row['Competition']:
        failed_df.at[index, 'Competition General'] = 'World Cup Qualifiers, Europe'
    if 'World Cup Qualifiers, Asia' in row['Competition']:
        failed_df.at[index, 'Competition General'] = 'World Cup Qualifiers, Asia'
    if 'World Cup Qualifiers, Africa' in row['Competition']:
        failed_df.at[index, 'Competition General'] = 'World Cup Qualifiers, Africa'
    if 'CONCACAF' in row['Competition']:
        failed_df.at[index, 'Competition General'] = 'World Cup Qualifiers, CONCACAF'
    if 'CONMEBOL' in row['Competition']:
        failed_df.at[index, 'Competition General'] = 'World Cup Qualifiers, CONMEBOL'
    if 'African Cup of Nations' in row['Competition']:
        failed_df.at[index, 'Competition General'] = 'African Cup of Nations'
    if 'Olympics' in row['Competition']:
        failed_df.at[index, 'Competition General'] = 'Olympics'
    if 'Gold Cup' in row['Competition']:
        failed_df.at[index, 'Competition General'] = 'Gold Cup'
    if 'World Cup Intercontinental Playoffs' in row['Competition']:
        failed_df.at[index, 'Competition General'] = 'World Cup Intercontinental Playoffs'
    if 'U-20 World Cup' in row['Competition']:
        failed_df.at[index, 'Competition General'] = 'U-20 World Cup'

In [62]:
combined_df.to_csv('combined_DF_terrikon_removed_IDs_FIXED.csv')

In [64]:
len(combined_df)

56928

In [61]:
failed_df.to_csv('failed_df_terrikon_removed_IDs_FIXED.csv')